# Basic fitting for hyperfine beat (stage 1 bootstrap)

From prior work and data:

- Forbes, R. et al. (2018) ‘Quantum-beat photoelectron-imaging spectroscopy of Xe in the VUV’, Physical Review A, 97(6), p. 063417. Available at: https://doi.org/10.1103/PhysRevA.97.063417. arXiv: http://arxiv.org/abs/1803.01081, Authorea (original HTML version): https://doi.org/10.22541/au.156045380.07795038
- Data (OSF): https://osf.io/ds8mk/
- [Quantum Metrology with Photoelectrons (Github repo)](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons), particularly the [Alignment 3 notebook](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons/blob/master/Alignment/Alignment-3.ipynb). Functions from this notebook have been incorporated in the current project, under `qbanalysis.hyperfine`.

For basic fitting, try a stage 1 style bootstrap. In this case, set (arbitrary) parameters per final state for the probe, and fit these plus the hyperfine beat model parameters. This should allow for a match to a single set of hyperfine parameters for all observables.

- 14/06/24: basic fit for L=4/ROI-0 data working with Scipy. Next should add ionization model and use all states...
   - Xarray wrapper may be neater? See https://docs.xarray.dev/en/latest/generated/xarray.DataArray.curvefit.html#xarray.DataArray.curvefit
   - 16/06/24: A,B param determination with Scipy.least_squares working. Seems like overkill, but other methods not very flexible? Currently pass Xarray data for calcs, with wrappers for Scipy. Quite annoying.
   - TODO: try PD-based calc, should actually be easier in this case.

## Setup fitting model

Follow the modelling notebook, but wrap functions for fitting.

New functions are in `qbanalysis.basic_fitting.py`.

### Imports

In [1]:
# Load packages
# Main functions used herein from qbanalysis.hyperfine
from qbanalysis.hyperfine import *
import numpy as np
from epsproc.sphCalc import setBLMs

from pathlib import Path

dataPath = Path('/tmp/xe_analysis')
# dataTypes = ['BLMall', 'BLMerr', 'BLMerrCycle']   # Read these types, should just do dir scan here.

# # Read from HDF5/NetCDF files
# # TO FIX: this should be identical to loadFinalDataset(dataPath), but gives slightly different plots - possibly complex/real/abs confusion?
# dataDict = {}
# for item in dataTypes:
#     dataDict[item] = IO.readXarray(fileName=f'Xe_dataset_{item}.nc', filePath=dataPath.as_posix()).real
#     dataDict[item].name = item

# Read from raw data files
from qbanalysis.dataset import loadFinalDataset
dataDict = loadFinalDataset(dataPath)

# Use Pandas and load Xe local data (ODS)
# These values were detemermined from the experimental data as detailed in ref. [4].
from qbanalysis.dataset import loadXeProps
xeProps = loadXeProps()

2024-06-17 15:57:25.965 | INFO     | qbanalysis.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 


* Setting plotter defaults with epsproc.basicPlotters.setPlotters(). Run directly to modify, or change options in local env.


* Set Holoviews with bokeh.
* pyevtk not found, VTK export not available. 
2024-06-17 15:57:31.570 | INFO     | qbanalysis.hyperfine:<module>:28 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-17 15:57:31.637 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_cycleSummed_rot90_quad1_ROI_results_with_FT_NFFT1024_hanningWindow_270717.mat.
2024-06-17 15:57:31.683 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_allCycles_ROIs_with_FTs_NFFT1024_hanningWindow_270717.mat.
2024-06-17 15:57:31.995 | INFO     | qbanalysis.dataset:loadFinalDataset:220 - Processed data to Xarray OK.
2024-06-17 15:57:32.035 | INFO     | qbanalysis.dataset:loadXeProps:71 - Loaded Xe data from /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/dataLocal/Xe_data_table_fixedFractions.ods.

**Xe measured level splittings and the hyperfine constants.**
Statistical uncer

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

### Init parameters & test

Here use `xeProps` to set and define fit paramters. Note in the original work the splittings were determined by FT of the data, and A, B parameters via Eqn. 2 therein.

TODO: may want to use lmfit here for more flexibility.

In [2]:
# Set splittings
fitParamsCol = 'Splitting/cm−1'
xePropsFit = xeProps.copy()

xeSplittings = xePropsFit[fitParamsCol].to_numpy()

In [3]:
# # Test beat model with changed params...
# xeSplittings = np.random.randn(4)
# xeSplittings

In [4]:
# xePropsFit[fitParamsCol] = 0.1*np.abs(xeSplittings)
# xePropsFit

In [5]:
# Test beat model with changed params...

# Set arb params
xeSplittings = np.random.randn(4)
xePropsFit[fitParamsCol] = 0.1*np.abs(xeSplittings)

# Compute model with new params
modelDict = computeModel(xePropsFit)
modelDictSum, modelDA = computeModelSum(modelDict)

# Plot model
plotOpts = {'width':800}
modelDA = stackModelToDA(modelDictSum)
plotHyperfineModel(modelDA, **plotOpts).opts(title="Isotope comparison + sum")

:Overlay
   .NdOverlay.I  :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (sum,sum_std)

## Run fits with Scipy Least Squares

Use the wrapper :py:func:`qbanalysis.basic_fitting.calcFitModel()` with `scipy.optimize.least_squares`. The wrapper uses `computeModelSum()` as above, and computes the residuals.

For the basic case, no ionization model is included, so this fit is only to see how well the form of the hyperfine beat can be matched to the  $K=4$ case for ROI 0, and how much the level splittings are modified from the previous case (determined by FT).

In [6]:
# Import functions
from qbanalysis.basic_fitting import *
import scipy

#*** Init params - either random or from previous best
# NOTE: this needs to be a 1D Numpy array.
# x0 = np.abs(np.random.random(4))  # Randomise inputs

# Seed with existing params - note this can't be Uncertainties objects
xePropsFit = xeProps.copy()
x0 = unumpy.nominal_values(xePropsFit[fitParamsCol].to_numpy())  # Test with previous vals

#*** Run a fit
fitOut = scipy.optimize.least_squares(calcBasicFitModel, x0, bounds = (0.01,0.5), verbose = 2,
                                      kwargs = {'xePropsFit':xePropsFit, 'dataDict':dataDict})
fitOut.success

2024-06-17 15:57:34.841 | INFO     | qbanalysis.basic_fitting:<module>:21 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.8508e-04                                    2.08e-02    
       1              2         4.0425e-05      1.45e-04       2.74e-03       3.53e-03    
       2              3         2.5587e-05      1.48e-05       1.63e-03       7.16e-04    
       3              4         2.3896e-05      1.69e-06       1.12e-03       2.73e-04    
       4              9         2.3837e-05      5.90e-08       9.90e-04       1.06e-04    
       5             11         2.3743e-05      9.33e-08       4.70e-04       2.70e-05    
       6             13         2.3727e-05      1.64e-08       2.33e-04       9.02e-06    
       7             15         2.3725e-05      1.51e-09       1.17e-04       1.75e-06    
       8             

True

In [7]:
# Using Scipy, the fit details are in fitOut, and results in fitOut.x
fitOut.x

array([0.29126303, 0.08466675, 0.1411    , 0.22949392])

In [8]:
fitOut

     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [ 8.541e-07  2.215e-05 ...  3.658e-04  5.021e-04]
           x: [ 2.913e-01  8.467e-02  1.411e-01  2.295e-01]
        cost: 2.3725378584475683e-05
         jac: [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
               [-9.299e-04  2.320e-04  0.000e+00 -5.137e-04]
               ...
               [ 5.836e-01 -3.171e-01  0.000e+00  2.142e-01]
               [ 7.487e-01 -3.187e-01  0.000e+00  9.719e-02]]
        grad: [-4.296e-09 -5.315e-08  0.000e+00  4.667e-09]
  optimality: 2.2076459731634488e-08
 active_mask: [0 0 0 0]
        nfev: 23
        njev: 12

In [9]:
# Check results - run model again with best fits
xePropsFit, modelFit, modelFitSum, modelIn, dataIn, res = calcBasicFitModel(fitOut.x, xePropsFit, dataDict, fitFlag=False)

# Fitted model & components
# (plotHyperfineModel(modelFit['129Xe'], **plotOpts) * plotHyperfineModel(modelFit['131Xe'], **plotOpts) * plotHyperfineModel(modelFitSum, **plotOpts)).opts(title="Isotope comparison + sum")

# Compare fit results with dataset
from qbanalysis.plots import plotFinalDatasetBLMt
# plotFinalDatasetBLMt(**dataDict, **plotOpts) * plotHyperfineModel(modelFitSum, **plotOpts).select(K=2).opts(**plotOpts)
(plotHyperfineModel(modelFitSum, **plotOpts,).select(K=2).opts(**plotOpts) * plotFinalDatasetBLMt(**dataDict, **plotOpts).select(l=4)).opts(title="Fit (K=2) plus data (l=4)")


:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II  :NdOverlay   [K,Q]
      :Curve   [t]   (sum,sum_std)
   .NdOverlay.III :NdOverlay   [ROI]
      :Spread   [t]   (BLM per cycle,BLM per cycle_std)
   .NdOverlay.IV  :NdOverlay   [ROI]
      :Curve   [t]   (BLM input 0)

In [10]:
# Check new results vs. reference case...
compareResults(xeProps, xePropsFit)

original       fit      diff
Isotope I   F   F′                                      
129     0.5 0.5 1.5  0.2863+/-0.0005  0.291263 -0.004963
131     1.5 1.5 0.5  0.0855+/-0.0010  0.084667  0.000833
            2.5 1.5  0.1411+/-0.0029  0.144827 -0.003727
                0.5  0.2276+/-0.0029  0.229494 -0.001894

Here we can see that - as expected - the fit is pretty good for the $l=4$, $ROI=0$ data. The fitted values are slightly different to the previous results (obtained via FT of the time-domain data).

## Determine A & B parameters

To further compare these new splittings with the previous results and literature, the A & B hyperfine parameters can be determined.

From the measurements, the hyperfine coupling constants can be determined by fitting to the usual form (see, e.g., ref. \cite{D_Amico_1999}):
\begin{equation}
\Delta E_{(F,F-1)}=AF+\frac{3}{2}BF\left(\frac{F^{2}+\frac{1}{2}-J(J+1)-I(I+1)}{IJ(2J-1)(2I-1)}\right)
\end{equation}

Note, for $^{129}\rm{Xe}$, $\Delta E_{(F,F-1)}=AF$ only ($B=0$).

In [11]:
extractABParams(xePropsFit)

/opt/conda/lib/python3.10/site-packages/scipy/optimize/_lsq/least_squares.py:111: UserWarning: Setting `gtol` below the machine epsilon (2.22e-16) effectively disables the corresponding termination condition.
  warn("Setting `{}` below the machine epsilon ({:.2e}) effectively "
/home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/qbanalysis/basic_fitting.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data129.loc[cols,'A/MHz'] = data129.loc[cols,'Splitting/cm−1']/data129.loc[cols,'F′'] * cmToMHz * phase


,Isotope,I,F,F′,A/MHz,B/MHz,Splitting/cm−1,dE
0,129,0.5,0.5,1.5,-5821.230642,nan+/-nan,0.291263,0.291263
1,131,1.5,1.5,0.5,1729.302016,37.13433,0.084667,0.084667
2,131,1.5,2.5,1.5,1729.302016,37.13433,0.144827,0.144828
3,131,1.5,2.5,0.5,1729.302016,37.13433,0.229494,0.229495


In [12]:
xePropsFit

A/MHz      B/MHz  Splitting/cm−1
Isotope I   F   F′                                       
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan        0.291263
131     1.5 1.5 0.5  1697+/-30     -8+/-7        0.084667
            2.5 1.5  1697+/-30     -8+/-7        0.144827
                0.5  1697+/-30     -8+/-7        0.229494

In [13]:
# # Fix splitting value (derived) - NOW IN MAIN ROUTINE
# iso=131
# xePropsFit.loc[(iso,1.5,2.5,1.5), dataCol] = xePropsFit.loc[(iso,1.5,2.5,0.5), dataCol] - xePropsFit.loc[(iso,1.5,1.5,0.5), dataCol]
# xePropsFit

In [14]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

### Determine A & B parameters

Previously fitted in Matlab with cffit tool, see `jake-home/tmp/xe_analysis_2024_scratch/Xe_hyperfine_cftool_fit_code_041217.m`.

From the measurements, the hyperfine coupling constants can be determined by fitting to the usual form (see, e.g., ref. \cite{D_Amico_1999}):
\begin{equation}
\Delta E_{(F,F-1)}=AF+\frac{3}{2}BF\left(\frac{F^{2}+\frac{1}{2}-J(J+1)-I(I+1)}{IJ(2J-1)(2I-1)}\right)
\end{equation}

Note, for $^{129}\rm{Xe}$, $\Delta E_{(F,F-1)}=AF$ only ($B=0$).

### Quick test with sklearn (linear)

In [ ]:
# Xe129, dE(F,F-1)=AF
# xePropsFit[[xePropsFit.xs((129)),'A/MHz']]  #['A/MHz']= 30

cmToMHz = 29979.2458
dataCol = 'Splitting/cm−1'
qnDF = xePropsFit.loc[(129)].index.to_frame()  # Convert to DF for easy Q.N./index value retrieval
xePropsFit.loc[(129), 'A/MHz'] = (xePropsFit.loc[(129), dataCol]/qnDF['F′']).values * cmToMHz
xePropsFit                                                                                                         

In [ ]:
xePropsFit.xs(131)[dataCol]

In [ ]:
xePropsFit.xs(iso)[dataCol].to_numpy()

In [ ]:
qnDF['F'].to_numpy()

In [ ]:
# Xe131
iso=131
qnDF = xePropsFit.loc[(iso)].index.to_frame()  # Convert to DF for easy Q.N./index value retrieval

# Fix splitting value (derived)
xePropsFit.loc[(iso,1.5,2.5,1.5), dataCol] = xePropsFit.loc[(iso,1.5,2.5,0.5), dataCol] - xePropsFit.loc[(iso,1.5,1.5,0.5), dataCol]
xePropsFit


# Quick test with linear regression
# https://realpython.com/linear-regression-in-python/
# Probably want scipy.curve_fit here though, with function defined.
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Quick test, only need 2 of these values, but should be more careful here...!
x=qnDF['F'][0:2].to_numpy().reshape(-1,1)
y=xePropsFit.xs(iso)[dataCol][0:2].to_numpy()
model.fit(x,y)
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

# TODO: check old work for method (not in Matlab code...?)
# TODO: other methods? Scipy or Xarray...?
# https://docs.xarray.dev/en/latest/generated/xarray.DataArray.curvefit.html#xarray.DataArray.curvefit

### v3 - just use PD...

In [ ]:
xePropsFit

In [ ]:
# Now in basic_fitting code

# # xeProps.unstack(['I','F','F′'])  #['I']
# dataPD = xeProps.reset_index()  # Unstack to cols for calculation

# A = 5700
# B = 10

# def dEcalc(dataInPD, A, B):
#     """
#     Calculate dE from A & B
    
#     The hyperfine coupling constants can be determined by fitting to the usual form (see, e.g., ref. \cite{D_Amico_1999}):
#     \begin{equation}
#     \Delta E_{(F,F-1)}=AF+\frac{3}{2}BF\left(\frac{F^{2}+\frac{1}{2}-J(J+1)-I(I+1)}{IJ(2J-1)(2I-1)}\right)
#     \end{equation}
    
#     NOTE: units currently set for return values only.
#     NOTE: this version assumes PD dataframe input. Multiindex including [Isotope,I,F], or columns OK.
    
#     """
#     # Set units
#     units = 'cm-1'
#     cmToMHz = 29979.2458

#     # Set J
#     J=1
    
#     # Set data - note reset index to just use col values
#     dataPD = dataInPD.copy()
#     if 'I' not in dataPD.columns:
#         dataPD = dataPD.reset_index()
        
        
#     I=dataPD['I']
#     c1=0.5-J*(J+1)-I*(I+1)
#     c2=I*J*(2*J-1)*(2*I-1)
#     F = dataPD['F']
#     t1 = A*F 
#     t2 = (3/2)*B*((F**2 + c1)/c2)
#     t2 = t2.fillna(0)

#     dataOut = dataPD.copy()
    
#     #*** Set outputs by isotope and dF
#     # Default case, t1 only
#     dataOut['dE'] = t1
    
#     # For 131Xe, use t1+t2
#     dataOut.loc[dataOut['Isotope']==131,'dE'] = (t1+t2)
    
#     # ...and replace cases with dF > 1
#     dataOut.loc[np.abs(dataOut['F'] - dataOut['F′'])>1, 'dE'] = np.nan
    
#     # Cheat here, and replace with sum for known case - should automate this!
#     # Note use of .values otherwise data with Uncertainties may not propagate correctly.
#     dataOut.loc[(dataOut['Isotope']==131) & (dataOut['F']==2.5) & (dataOut['F′']==0.5),'dE'] = dataOut[(dataOut['Isotope']==131) & (dataOut['F']==2.5) & (dataOut['F′']==1.5)]['dE'].values + dataOut[(dataOut['Isotope']==131) & (dataOut['F']==1.5) & (dataOut['F′']==0.5)]['dE'].values

    
#     # Set units
#     if units == 'cm-1':
#         dataOut['dE'] = dataOut['dE']*(1/cmToMHz)
    

#     return dataOut

# dataOut = dEcalc(xePropsFit, dataPD['A/MHz'], dataPD['B/MHz'])

# dataOut

In [ ]:
unumpy.nominal_values(dataOut['Splitting/cm−1'] - dataOut['dE'])

In [ ]:
dataOut

In [ ]:
dEcalcWrapperScipy([2000,30],xeDataInPD=data131)

In [ ]:
data131 = xePropsFit.reset_index()
data131 = data131.loc[data131['Isotope']==131]
data131

In [ ]:
# Now in basic fitting code

# def dEcalcWrapperScipy(x0,xeDataInPD=None):
#     """
#     Wrap dEcalc for Scipy least_squares...
#     """
    
#     dEOut = dEcalc(xeDataInPD, x0[0], x0[1])
    
#     res = ((dEOut['Splitting/cm−1'] - dEOut['dE'])**2) #.squeeze()
    
#     if unFlag:
#         res = unumpy.nominal_values(res)
    
#     return res

# # Set data to fit - NOTE 131 only!
# data131 = xePropsFit.reset_index()   # Use reset here, xePropsFit.xs(131) drops index
# data131 = data131.loc[data131['Isotope']==131]


# # x0in = np.random.rand(2)
# x0in = [2000,30]

# fitOut = scipy.optimize.least_squares(dEcalcWrapperScipy, x0in, bounds = ([0,-100],[2500,100]),
#                                       kwargs = {'xeDataInPD':data131},
#                                       verbose = 2,
#                                       xtol=1e-12,ftol=1e-12,gtol=1e-18)

In [ ]:
fitOut

In [ ]:
fitOut.x

In [ ]:
dOut = dEcalc(data131, *fitOut.x)
dOut

In [ ]:
dataCols = xePropsFit.reset_index()   # Use reset here, xePropsFit.xs(131) drops index
data129 = dataCols.loc[dataCols['Isotope']==129]

data129.append(dOut)  #.append

In [ ]:
import pandas as pd
def extractABParams(xePropsFit):
    """
    Determine A & B parameters from hyperfine level splittings.
    
    This runs a quick fit with Scipy
    """
    
    #*** Set data to fit - NOTE 131 only!
    dataCols = xePropsFit.reset_index()   # Use reset here, xePropsFit.xs(131) drops index
    data131 = dataCols.loc[dataCols['Isotope']==131]

    #*** Extract A,B for 131Xe using Scipy fitting
    # x0in = np.random.rand(2)
    x0in = [2000,30]

    fitOut = scipy.optimize.least_squares(dEcalcWrapperScipy, x0in, bounds = ([0,-100],[2500,100]),
                                          kwargs = {'xeDataInPD':data131},
                                          verbose = 0,
                                          xtol=1e-12,ftol=1e-12,gtol=1e-18)
    
    # Set final results
    dataFit = dEcalc(data131, *fitOut.x)
    
    # Add fitted results to table
    dataFit['A/MHz'] = fitOut.x[0]
    dataFit['B/MHz'] = fitOut.x[1]
    
    # Add 129 case back in
    # Note phase = -1 by convention, since F<F' - now included in dE calc directly
    # phase =-1
    data129 = dataCols.loc[dataCols['Isotope']==129]
    
    # This works, but always throwing PD warning?
    # data129.loc[data129['Isotope']==129,'A/MHz']= data129['Splitting/cm−1']/data129['F′'] * cmToMHz
    cols = data129['Isotope']==129
    data129.loc[cols,'A/MHz'] = data129.loc[cols,'Splitting/cm−1']/data129.loc[cols,'F′'] * cmToMHz 
    # data129.loc[cols,'dE'] = data129.loc[cols,'F′'] * data129.loc[cols,'A/MHz'] * 1/cmToMHz
    
    data129 = dEcalc(data129, data129['A/MHz'], np.nan)
    
    # data129.loc[data129.columns['A/MHz']]
    # dF = xeData.F - xeData['F′']
    dataOut = pd.concat([data129,dataFit])
    
    return dataOut

In [ ]:
dataCols['F']- dataCols['F′']

In [ ]:
extractABParams(xePropsFit)

In [ ]:
Iind = dataCols['Isotope']
dataCols.loc[Iind==129,'A/MHz']=data129['Splitting/cm−1']/data129['F′']

In [ ]:
dataCols

In [ ]:
data129.loc[,'A/MHz']=data129['Splitting/cm−1']/data129['F′']  #).values
data129

In [ ]:
dataOut.columns

In [ ]:
np.abs(dataOut['F'] - dataOut['F′'])>1

In [ ]:
dataOut[np.abs(dataOut['F'] - dataOut['F′'])>1]

### Use Xarray wrapper - should be easier... V2: extract from PD, then XR

In [ ]:
# xeData = xePropsFit.to_xarray()

# xePropsFit.s
dataCol = 'Splitting/cm−1'
xePropsFit.xs(131)[dataCol]
# unstack .to_xarray()

In [ ]:
# dEXR.loc[{'Isotope':131,'F':2.5,'F′':0.5, 'I':1.5}]

In [ ]:
# NEW APPROACH - set up specific DA for fitting...
# THIS will return required vals only.
xeSubSel = xePropsFit.xs(131)[dataCol][0:2].droplevel('F′').to_xarray()  # for f' too?
A = xePropsFit.xs(131)['A/MHz'][0:2].droplevel('F′').to_xarray()
B = xePropsFit.xs(131)['B/MHz'][0:2].droplevel('F′').to_xarray()

dESubSel = dEv2(xeSubSel,A,B)
dESubSel

In [ ]:
xeSubSel.squeeze().hvplot()

In [ ]:
xeSubSel.squeeze() - dESubSel

In [ ]:
# With Scipy wrapper... test return OK...
print(dEv2WrapperScipy([2000,0],xeDataIn=xeSubSel))
print(dEv2WrapperScipy([1697,0],xeDataIn=xeSubSel))
print(dEv2WrapperScipy([1697,-8],xeDataIn=xeSubSel))

In [ ]:
# # Ah, can't pass additional args to curvefit in any case...????
# dsFit = dESubSel.curvefit(

#     coords=dESubSel.F,
    
#     # reduce_dims="I",

#     func=dEv2Wrapper,

#     bounds={"A": (1000, 2000), "B": (-15, 15)},
    
#     param_names=["A","B"],
    
#     kwargs={'xeDataIn':dESubSel}

# )

# USE WRAPPER FOR SCIPY DIRECTLY... WORKING...
#
# Defaults give [1718.15579161,   -2.19752871], [1663.67627212,    4.42848128], [ 1.69247409e+03, -1.49524345e-01]
# Seems more dependent on x0in than tolerances...? Could do with more points?
# Sets B to ~0 if bounded at 0, so -ve definitely good it seems, although not well defined?
#
# UPDATE: had mixed new and old values... now fixed...
#  [1729.31918052,   37.13028214]  # Seems pretty consistent vs. x0in.
#
# x0in = [2000,0]
# x0in = [1500, 0]
x0in = np.random.rand(2)

fitOut = scipy.optimize.least_squares(dEv2WrapperScipy, x0in, bounds = ([0,-100],[2500,100]),
                                      kwargs = {'xeDataIn':xeSubSel},
                                      verbose = 2,
                                      xtol=1e-12,ftol=1e-12,gtol=1e-18)

In [ ]:
fitOut

In [ ]:
fitOut.x

In [ ]:
# 17/06/24: now in basic_fitting.py

# # V2, testing subselected PD array > Xr
# # Calculate dE for Xarray input - in this case all coords should match in size...
# # A,B can be Xarray or scalar
# # def dEv2(xeDataIn, A, B, units = 'cm-1'):
# def dEv2(xeDataIn, A, B):
#     """
#     the hyperfine coupling constants can be determined by fitting to the usual form (see, e.g., ref. \cite{D_Amico_1999}):
#     \begin{equation}
#     \Delta E_{(F,F-1)}=AF+\frac{3}{2}BF\left(\frac{F^{2}+\frac{1}{2}-J(J+1)-I(I+1)}{IJ(2J-1)(2I-1)}\right)
#     \end{equation}
    
#     NOTE: units currently set for return values only.
#     """
#     # for iso in xeData.Isotope:
#     #     print(item)
#     units = 'cm-1'
#     cmToMHz = 29979.2458
    
#     # Isotope terms
#     J=1
#     I=xeDataIn.I
#     c1=0.5-J*(J+1)-I*(I+1)
#     c2=I*J*(2*J-1)*(2*I-1)
    
#     # A/B terms
#     F = xeDataIn.F
#     # F = xeData['F′']  # TODO: fix 129 ordering, needs F,F' swapped! (Or enforce selection here...)
#                         # Or swap on max value, or unique values... 
#                         # Or check on dF, xeData.F - xeData['F′'] ...?
#     # This works, but have some redundant values still
#     # Ffixed = xr.where(xeData.F > xeData['F′'], xeData.F, xeData['F′'])  # Check greater
#     # F = Ffixed[:,1]
    
#     # Try unique vals only... Breaks 129 case...
#     # F = xeData.F.where(xeData.F > 1)
    
#     # Deltas... filter on these at return?
#     # dF = xeData.F - xeData['F′']
    
    
#     t1 = A*F  #* np.sign(dF)
    
#     # For Xr case avoid Nan propagation
#     if isinstance(B, xr.DataArray):
#         if unFlag:
#             B = xrUnFillna(B)
#         else:
#             B = B.fillna(0)
        
        
#     t2 = (3/2)*B*((F**2 + c1)/c2)
#     t2 = t2.fillna(0)
    
#     # return t1,t2,t1+t2
    
#     if units == 'MHz':
#         dEout = t1+t2
#     elif units == 'cm-1':
#         dEout = (t1+t2)*(1/cmToMHz)

#     # Check allowed terms...?
#     # dEout = dEout.where(np.abs(dF)<2,np.nan)
        
        
#     # TODO: general fix for F-F' > 1..?
#     # dEXR.where(dEXR.F - dEXR['F′'] > 1)
#     # Quick fix here for Xe131 case only
#     # dEout.sel({'F':
#     # dEout.loc[{'Isotope':131,'F':2.5,'F′':0.5, 'I':1.5}] = dEout.sel({'Isotope':131,'F':1.5,'F′':0.5, 'I':1.5}) + dEout.sel({'Isotope':131,'F':2.5,'F′':1.5, 'I':1.5})
    
#     if unFlag:
#         dEout.values = unumpy.nominal_values(dEout)
        
#     return dEout


# def dEv2Wrapper(xeDataInNP, A, B, xeDataIn):
#     """
#     Thin wrapper for xr.curvefit.
    
#     Just swap NP data as passed for XR data to use existing function
#     """
    
#     dEOut = dEv2(xeDataIn, A, B)
    
#     return dEout

    
# def dEv2WrapperScipy(x0,xeDataIn=None):
#     """
#     ... and wrap for Scipy least_squares...
#     """
    
#     dEOut = dEv2(xeDataIn, x0[0], x0[1])
    
#     res = ((xeDataIn - dEOut)**2).squeeze()
    
#     return res.values

    
# def xrUnFillna(xrData):
#     """
#     Implement xr.fillna for Uncertainties data types.
#     """
    
#     return xrData.where(~unumpy.isnan(xrData),0)

### Use Xarray wrapper - should be easier... V1: pure Xr, lots of redun

In [ ]:
xeData = xePropsFit.to_xarray()
xeData

In [ ]:
xeData.F

In [ ]:
# 17/06/24: now in basic_fitting.py

# # V1, assumes full PD array > Xr
# # Calculate dE for Xarray input - in this case all coords should match in size...
# # A,B can be Xarray or scalar
# def dE(xeDataIn, A, B, units = 'cm-1'):
#     """
#     the hyperfine coupling constants can be determined by fitting to the usual form (see, e.g., ref. \cite{D_Amico_1999}):
#     \begin{equation}
#     \Delta E_{(F,F-1)}=AF+\frac{3}{2}BF\left(\frac{F^{2}+\frac{1}{2}-J(J+1)-I(I+1)}{IJ(2J-1)(2I-1)}\right)
#     \end{equation}
    
#     NOTE: units currently set for return values only.
#     """
#     # for iso in xeData.Isotope:
#     #     print(item)
#     cmToMHz = 29979.2458
    
#     # Isotope terms
#     J=1
#     I=xeDataIn.I
#     c1=0.5-J*(J+1)-I*(I+1)
#     c2=I*J*(2*J-1)*(2*I-1)
    
#     # A/B terms
#     # F = xeDataIn.F
#     # F = xeData['F′']  # TODO: fix 129 ordering, needs F,F' swapped! (Or enforce selection here...)
#                         # Or swap on max value, or unique values... 
#                         # Or check on dF, xeData.F - xeData['F′'] ...?
#     # This works, but have some redundant values still
#     Ffixed = xr.where(xeData.F > xeData['F′'], xeData.F, xeData['F′'])  # Check greater
#     F = Ffixed[:,1]
    
#     # Try unique vals only... Breaks 129 case...
#     # F = xeData.F.where(xeData.F > 1)
    
#     # Deltas... filter on these at return?
#     dF = xeData.F - xeData['F′']
    
    
#     t1 = A*F* np.sign(dF)
    
#     # For Xr case avoid Nan propagation
#     if isinstance(B, xr.DataArray):
#         if unFlag:
#             B = xrUnFillna(B)
#         else:
#             B = B.fillna(0)
        
        
#     t2 = (3/2)*B*((F**2 + c1)/c2)
#     t2 = t2.fillna(0)
    
#     # return t1,t2,t1+t2
    
#     if units == 'MHz':
#         dEout = t1+t2
#     elif units == 'cm-1':
#         dEout = (t1+t2)*(1/cmToMHz)

#     # Check allowed terms...?
#     dEout = dEout.where(np.abs(dF)<2,np.nan)
        
        
#     # TODO: general fix for F-F' > 1..?
#     # dEXR.where(dEXR.F - dEXR['F′'] > 1)
#     # Quick fix here for Xe131 case only
#     # dEout.sel({'F':
#     # dEout.loc[{'Isotope':131,'F':2.5,'F′':0.5, 'I':1.5}] = dEout.sel({'Isotope':131,'F':1.5,'F′':0.5, 'I':1.5}) + dEout.sel({'Isotope':131,'F':2.5,'F′':1.5, 'I':1.5})
    
    
#     return dEout

# def xrUnFillna(xrData):
#     """
#     Implement xr.fillna for Uncertainties data types.
#     """
    
#     return xrData.where(~unumpy.isnan(xrData),0)

In [ ]:
Ffixed = xr.where(xeData.F > xeData['F′'], xeData.F, xeData['F′'])
Ffixed[:,1]

In [ ]:
xeData.F.where(xeData.F > 1)

In [ ]:
dF = xeData.F - xeData['F′']
xeData.where(np.abs(dF)<2,0)
dF

In [ ]:
xeData.F

In [ ]:
dE(xeData,3,10)

In [ ]:
isinstance(xeData['B/MHz'], xr.DataArray)

In [ ]:
dEXR = dE(xeData, xeData['A/MHz'], xeData['B/MHz'])

dEXR  #.sel(Isotope=129)

In [ ]:
# Reduce to only required dim for fitting...
dEsub = dEXR.loc[{'Isotope':131,'F':[1.5,2.5],'I':1.5}]  #.dropna()

# dEsub.hvplot.line(x='F')

In [ ]:
# dEXR.sel({'Isotope':131,'F':2.5,'F′':0.5, 'I':1.5}) = dEXR.sel({'Isotope':131,'F':1.5,'F′':0.5, 'I':1.5}) + dEXR.sel({'Isotope':131,'F':2.5,'F′':1.5, 'I':1.5})
dEXR.loc[{'Isotope':131,'F':2.5,'F′':0.5, 'I':1.5}] = dEXR.sel({'Isotope':131,'F':1.5,'F′':0.5, 'I':1.5}) + dEXR.sel({'Isotope':131,'F':2.5,'F′':1.5, 'I':1.5})

In [ ]:
dEXR.loc[{'Isotope':131, 'I':1.5}].drop('F′')

In [ ]:
Ffixed

In [ ]:
dEXR.loc[{'Isotope':131,'F':2.5,'F′':0.5, 'I':1.5}]

In [ ]:
dEXR.sel({'Isotope':131,'I':1.5},drop=True) #.flatten()  #.dropna()

In [ ]:
dEXR.where(dEXR.F - dEXR['F′'] > 1)

In [ ]:
dE(xeData,0,0)

In [ ]:
F = xeData.F
xeData['A/MHz'] * F

In [ ]:
B = xeData['B/MHz']

J=1
I=xeData.I
c1=0.5-J*(J+1)-I*(I+1)
c2=I*J*(2*J-1)*(2*I-1)

# For Xr case avoid Nan propagation
if isinstance(B, xr.DataArray):
    B = B.fillna(0)

t2 = (3/2)*B*((F**2 + c1)/c2)
t2 = t2.fillna(0)
t2

In [ ]:
# B
# uncertainties.umath.isnan(B)
unumpy.nominal_values(B)

In [ ]:
xeData.sel(Isotope=131)['F']

In [ ]:
xeData.where(F>0.5)

In [ ]:
xeData.sel(Isotope=129)['F′']

In [ ]:
umath.isnan

In [ ]:
from uncertainties import umath
# umath.isnan(B)
# B.pipe(umath.isnan)
unumpy.isnan(B)

In [ ]:
B.where(~unumpy.isnan(B),0)

In [ ]:
import hvplot.pandas
xePropsFit['Splitting/cm−1'].plot()

In [ ]:
xePropsFit.index.values

In [ ]:
# xePropsFit['Splitting/cm−1'].hvplot()  # No multindex support...?

# Ah, probably need to flatten - should have code elsewhere for this.
# Quick go per https://stackoverflow.com/questions/74860179/hvplot-interactive-pd-dataframe-with-multiindex
# Fails for Series... except with names only... can't recall how to fix this right now...
dataPlot = xePropsFit['Splitting/cm−1'].set_axis(map(" ".join, xePropsFit.index.names), axis=0)
# dataPlot = xePropsFit['Splitting/cm−1'].set_axis(map(" ".join, qnDF.columns), axis=0)

# qnDF['F']
# dataPlot = xePropsFit.set_axis(map(" ".join, xePropsFit.index.values), axis=1)
dataPlot.hvplot()

In [ ]:
(xePropsFit.loc[(129), 'Splitting/cm−1']/xePropsFit.loc[(129)].index.to_frame()['F′']).values

In [ ]:
# xePropsFit.loc[(129)].index[0]
xePropsFit.loc[(129)].index.to_frame()['F']

In [ ]:
xeProps

## SCRATCH

In [ ]:
xeProps['A/MHz']

In [ ]:
xeProps.xs((131,1.5,1.5,0.5))

In [ ]:
xeProps.to_xarray()